In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import math
from my_parameters import ProblemParameters
EPS = 1e-6

In [ ]:
params = ProblemParameters(d=50, r=25)
d=params.d # 产品数量
R = params.r   # 地区数量（大幅增加）
# z_max = params.z_max  # 单地区最大分配量（提高）

lambdas=params.lambdas

# (2) 供应量 S（基于需求，但引入短缺/过剩）
S =params.S

# (3) 地区容量 K（模拟不同地区仓储限制）
K =params.K

# 预先计算每个产品在每个地区的销量和对应的概率
prob_matr, y_matr= params.compute_d_prob()

### 子问题

In [ ]:
def create_pwl(y_list, prob_list,j,r, step=3):
    z_vals=list(np.arange(0,min(K[r],S[j]),step))
    y_vals=[]
    for z in z_vals:
        y_expec=sum(min(z,y_list[k])*prob_list[k] for k in range(len(y_list)))
        y_vals.append(y_expec)
    print(f'{j}类产品在{r}地的分段为：',z_vals)
    print(y_vals)
    return z_vals, y_vals

In [ ]:
def solve_subproblem(j, duals_cap, duals_conv):
    """智能选择精确或近似方法的子问题求解"""
    model = gp.Model(f"subproblem_hybrid_{j}")
    model.setParam("OutputFlag", 0)
    
    z = model.addVars(R, name="z")
    for r in range(R):
        z[r].ub = K[r]  # 显式设置每个地区的上界
    
    # 辅助变量存储
    expect_vars = model.addVars(R, name="expect")
    
    # 为每个地区选择建模方法
    for r in range(R):
        y_list = y_matr[j, r]
        prob_list = prob_matr[j, r]
        
        if len(y_list) <= 20:  # 精确建模
            # 创建min(z[r],y)变量
            min_vars = model.addVars(len(y_list), lb=0, name=f"min_{r}")
            
            # 添加精确约束
            for idx, y in enumerate(y_list):
                # min(z,y) <= z
                model.addConstr(min_vars[idx] <= z[r], name=f"min_z_{r}_{idx}")
                # min(z,y) <= y
                model.addConstr(min_vars[idx] <= y, name=f"min_y_{r}_{idx}")

            # 期望值计算，即辅助变量关于概率的加权求和就是j类产品在r地区的期望销量
            model.addConstr(
                expect_vars[r] == gp.quicksum(prob_list[idx]*min_vars[idx] 
                                      for idx in range(len(y_list))),
                name=f"expect_exact_{r}"
            )
            
        else:  # PWL近似
            x_vals, y_vals = create_pwl(y_list, prob_list,j,r)
            model.addGenConstrPWL(z[r], expect_vars[r], x_vals, y_vals,
                                name=f"expect_pwl_{r}")
    
    # 供应约束，所生成列的可行性
    model.addConstr(z.sum('*') <= S[j], name="supply_constraint")
    
    # 目标函数，其实应该为检验数的相反数，因为问题是最大化
    obj = gp.quicksum(duals_cap[r] * z[r] for r in range(R)) - \
          gp.quicksum(expect_vars[r] for r in range(R)) + \
          duals_conv[j]
    model.setObjective(obj, GRB.MINIMIZE)
    model.write('subproblem.lp')
    # 求解
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        alloc = [int(round(z[r].X)) for r in range(R)]
        reduced_cost = model.ObjVal
        if reduced_cost < -1e-6:
            return alloc,  reduced_cost
    if model.status == GRB.INFEASIBLE:
        model.computeIIS()  # 计算不可行冲突约束
        model.write("infeasible.ilp")  # 输出不可行模型
        print("不可行约束见 infeasible.ilp 文件")

    return None,  None

### 主问题

In [ ]:
def build_master_problem(columns):
    m = gp.Model("master")
    m.setParam("OutputFlag", 0)
    m.setParam("NumericFocus", 1)  # 提高数值稳定性
    
    # 变量定义（显式设置上界）
    lambda_vars = {}
    convex_constrs = {}
    for j in columns.keys():
        for k, alloc in enumerate(columns[j]):
            lambda_vars[(j, k)] = m.addVar(lb=0, ub=1, name=f"lambda_{j}_{k}")

    # 凸组合约束
        convex_constrs[j] = m.addConstr(
            gp.quicksum(lambda_vars[(j, k)] for k in range(len(columns[j]))) == 1,
            name=f"convex_{j}"
        )

    # 地区容量约束
    cap_constrs = []
    for r in range(R):
        expr = gp.quicksum(
            alloc[r] * lambda_vars[(j, k)]
            for j in columns.keys()
            for k, alloc in enumerate(columns[j])#键j对应多个alloc
        )
        cap_constrs.append(m.addConstr(expr <= K[r], name=f"cap_{r}"))

    # 目标函数（向量化计算）
    obj_terms = []
    for j in columns.keys():
        for k, alloc in enumerate(columns[j]):
            term = 0
            for r in range(R):
                y_list = y_matr[j, r]
                prob_list = prob_matr[j, r]
                term += np.sum(np.minimum(alloc[r], y_list) * prob_list)  # 对第r个地区的需求和当前分配加权求和，并采用向量化加速
            obj_terms.append(term * lambda_vars[(j, k)])
    
    m.setObjective(gp.quicksum(obj_terms), GRB.MAXIMIZE)
    m.write('RLMP.lp')
    # 求解与诊断
    m.optimize()

    if m.status == GRB.OPTIMAL:
        duals = {
            'cap': [c.Pi for c in cap_constrs],
            'conv': {j: convex_constrs[j].Pi for j in convex_constrs}
        }
        return m, duals, m.objVal
    else:
        print(f"求解失败，状态码: {m.status}")
        if m.status == GRB.INFEASIBLE:
            m.computeIIS()
            m.write("infeasible.ilp")
        return None, {'cap': [0]*R, 'conv': {j:0 for j in columns}}, 0

### 创建初始解

In [ ]:
from collections import defaultdict

# 改进后的列存储结构
columns = defaultdict(list)  # 使用defaultdict自动初始化空列表
col_count = {}  # 记录每个产品已生成的列数

for j in range(d):
    col_count[j] = 0  # 初始化产品j的列计数器
    
    # 计算总需求强度
    total_lambda = sum(lambdas[j, r] for r in range(R))
    
    if total_lambda > 0:
        # 方案1：按需求比例分配（向下取整）
        alloc1 = [min(math.floor(S[j] * lambdas[j, r] / total_lambda), K[r]) for r in range(R)]
        
        # 方案2：方案1的减量版本（探索不同可能性）
        alloc2 = [min(max(0, alloc1[r] - 1), K[r]) for r in range(R)]  # 确保不小于0
    else:
        # 如果需求强度为零，则平均分配
        alloc1 = [min(math.floor(S[j] / R), K[r]) for r in range(R)]
        alloc2 = [min(max(0, alloc1[r] - 1), K[r]) for r in range(R)]
    
    # 存储列信息（使用更结构化的字典）
    columns[j].append(alloc1)
    col_count[j] += 1
    
    columns[j].append(alloc2)
    col_count[j] += 1

# 打印检查
print("产品列存储结构：")
for j in range(d):
    print(f"产品 {j} 的列：")
    print(columns[j])
print("\n列计数器状态：", col_count)
print("各产品的供应：",S)
print("地区容量：",K)
print(columns)

In [ ]:
MAX_ITER = 40
best_obj = -float('inf')
for it in range(MAX_ITER):
    print(f"\nIteration {it+1}")
    
    # 1. 求解主问题
    master, duals, obj_val = build_master_problem(columns)
    duals_cap=duals['cap']
    duals_conv=duals['conv']
    if master is None:
        print("Master problem infeasible.")
        break
        
    print(f"Current objective: {obj_val:.4f}")
    print(f"Dual prices (cap, conv): {duals_cap}, {duals_conv}")
    
    # 2. 检查是否收敛
    if abs(obj_val - best_obj) < 1e-6 and it > 0:
        print(f"Converged with objective {obj_val:.4f}.")
        break
    best_obj = obj_val
    
    # 3. 求解子问题
    improved = False
    for j in range(d):# 子问题返回分配方案和检验数
        alloc, reduced_cost = solve_subproblem(j, duals_cap, duals_conv)
        
        if reduced_cost is None:
            print(f"Product {j}: no feasible solution.")
            continue
            
        print(f"Product {j}: reduced cost = {reduced_cost:.6f}")
        
        # 4. 添加新列的条件：Reduced Cost < -EPS
        if reduced_cost < -EPS:
            k = col_count[j]
            columns[j].append(alloc)
            col_count[j] += 1
            improved = True
            print(f"  Added column for j={j}, k={k}: alloc={alloc}")
            break#一旦找到新列就直接加进主问题，再求解

    # 5. 终止条件：无改进列
    if not improved:
        print("No improving columns found. Stopping.")
        final_columns = columns
        break

print("\nFinal solution:")
for j in range(d):
    print(f"Product {j} columns:", columns[j])

In [ ]:
# 在主循环结束后添加：
if columns is not None:
    for j in range(d):
        lambda_vars = [v for v in master.getVars() if v.VarName.startswith(f"lambda_{j}_")]
        for v in lambda_vars:
            if v.X > 1e-6:  # 打印所有非零λ
                print(f"Product {j}, column {v.VarName}: λ = {v.X:.4f}")
                alloc = columns[j]
                print(f"  Allocation: {alloc}")

In [ ]:
# 4. 输出最终分配方案和最优目标函数值
if columns is not None:
    print("\n=== Final Allocation ===")
    for j in range(d):
        # 获取主问题中产品j的最优方案（λ_j ≈ 1的列）
        lambda_vars = [v for v in master.getVars() if v.VarName.startswith(f"lambda_{j}_")]
        optimal_col_idx = None
        for v in lambda_vars:
            if abs(v.X - 1.0) < 1e-6:  # 判断是否为选中的列（λ=1）
                optimal_col_idx = int(v.VarName.split("_")[-1])
                break
        
        if optimal_col_idx is not None:
            alloc = columns[j][optimal_col_idx]
            for r in range(R):
                if alloc[r] > 0:  # 仅输出>0的分配
                    print(f"Product {j} -> Region {r}: {alloc[r]} units")
    
    # 输出最优目标函数值
    print("\n=== Optimal Objective Value ===")
    print(f"Total Cost: {master.ObjVal}")
else:
    print("No feasible solution found.")